<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week09_Chavrusa_04/gemini_pydantic_more_robust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structure gemini outputs with Pydantic
- Pydantic 모델을 LLM(Gemini)의 출력 구조를 보장하는 도구로 사용하는 가장 강력하고 표준적인 방법


In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('gemini-key')

from google import genai
client = genai.Client(api_key=GEMINI_API_KEY)

In [5]:
from pydantic import BaseModel

class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]

response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "List a few asian soups recipes, a yummy description and list the ingredients",
)

print(response.text)

Here are a few popular Asian soup recipes, each with a yummy description and its key ingredients:

---

### 1. **Tom Yum Goong (Thai Hot & Sour Shrimp Soup)**

**Yummy Description:** Get ready for an explosion of vibrant flavors! This iconic Thai soup is a thrilling dance of hot, sour, salty, and a hint of sweet. Fragrant lemongrass, galangal, and kaffir lime leaves infuse a clear, fiery broth, studded with succulent shrimp and earthy mushrooms. It's incredibly aromatic, refreshingly zesty, and guaranteed to awaken your taste buds with its exhilarating spice and tangy kick.

**Key Ingredients:**

*   **Broth Base:** Chicken or vegetable broth
*   **Aromatics:** Fresh lemongrass (bruised), galangal (sliced), kaffir lime leaves (torn), bird's eye chilies (fresh or chili paste/Prik Pao)
*   **Protein:** Fresh large shrimp (peeled and de-veined)
*   **Vegetables:** Straw mushrooms or oyster mushrooms, cherry tomatoes (optional)
*   **Seasoning:** Fish sauce, fresh lime juice
*   **Garnish:

## 스키마를 이용한 LLM 출력 강제 구조화
- 프롬프트 내 지시(Text Prompting)에 의존하지 않고, Pydantic 모델을 JSON 스키마로 변환하여 Gemini API에 전달함으로써, 모델이 반드시 유효한 JSON 형식의 데이터를 생성하도록 강제

In [22]:
Recipe(recipe_name='Sample Recipe', description='This is a sample recipe.', ingredients=['ingredient1', 'ingredient2'])

Recipe(recipe_name='Sample Recipe', description='This is a sample recipe.', ingredients=['ingredient1', 'ingredient2'])

In [15]:

response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "List a few asian soups recipes, a yummy description and list the ingredients",
    config = {"response_mime_type": "application/json", # 💡 출력을 JSON으로 요청
              "response_schema": list[Recipe]}          # 💡 이 JSON 스키마를 따르도록 강제
)

print(response.text)


[{"recipe_name":"Tom Yum Goong","description":"A classic spicy and sour Thai soup, famous for its vibrant flavors, aromatic herbs, and plump shrimp. It's a tantalizing balance of hot, sour, salty, and sweet, often garnished with fresh cilantro.","ingredients":["Shrimp","Mushrooms","Lemongrass","Galangal","Kaffir lime leaves","Fish sauce","Lime juice","Chili paste (Nam Prik Pao)","Cilantro","Water or chicken broth"]},{"recipe_name":"Miso Soup","description":"A comforting and savory traditional Japanese soup made from dashi (a fish and kelp stock) and miso paste. It's often served with tofu and wakame seaweed, offering a delicate umami flavor.","ingredients":["Dashi (water, kombu, bonito flakes)","Miso paste","Tofu (silken or firm)","Wakame seaweed","Green onions"]},{"recipe_name":"Pho Bo (Vietnamese Beef Noodle Soup)","description":"Vietnam's beloved national dish, a fragrant and deeply flavorful beef noodle soup. It features a rich, clear broth simmered for hours with spices, served wi

In [16]:
type(response.text)

str

## simulate house prices

In [28]:
from typing import Literal

class Home(BaseModel):
    price: int
    monthly_fee: int
    living_area: float
    number_rooms: int
    type: Literal["apartment", "house"]
    address: str



response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = """List 50 apartments and houses in Sweden with their monthly_fee, their price, living area,
    number of rooms, address, type if it is apartment or house.
    All currencies are in SEK.""",
    config = {"response_mime_type": "application/json",
              "response_schema": list[Home]}
)


homes = response.parsed
homes

[Home(price=2500000, monthly_fee=3500, living_area=75.5, number_rooms=3, type='apartment', address='Storgatan 12, Stockholm'),
 Home(price=4800000, monthly_fee=2500, living_area=120.0, number_rooms=5, type='house', address='Västra Vägen 8, Göteborg'),
 Home(price=1800000, monthly_fee=2800, living_area=50.0, number_rooms=2, type='apartment', address='Lilla Allén 5, Malmö'),
 Home(price=3200000, monthly_fee=1800, living_area=90.0, number_rooms=4, type='house', address='Gröna Gatan 15, Uppsala'),
 Home(price=2100000, monthly_fee=4200, living_area=65.0, number_rooms=3, type='apartment', address='Norra Parkvägen 7, Lund'),
 Home(price=5500000, monthly_fee=3000, living_area=150.0, number_rooms=6, type='house', address='Skogsvägen 22, Linköping'),
 Home(price=1600000, monthly_fee=3100, living_area=45.0, number_rooms=2, type='apartment', address='Östra Hamngatan 3, Helsingborg'),
 Home(price=3900000, monthly_fee=2200, living_area=105.0, number_rooms=4, type='house', address='Solrosvägen 10, Ör

In [30]:
len(homes)

50

In [35]:
import pandas as pd

df = pd.DataFrame([home.__dict__ for home in homes])
df.head()

,price,monthly_fee,living_area,number_rooms,type,address
0,2500000,3500,75.5,3,apartment,"Storgatan 12, Stockholm"
1,4800000,2500,120.0,5,house,"Västra Vägen 8, Göteborg"
2,1800000,2800,50.0,2,apartment,"Lilla Allén 5, Malmö"
3,3200000,1800,90.0,4,house,"Gröna Gatan 15, Uppsala"
4,2100000,4200,65.0,3,apartment,"Norra Parkvägen 7, Lund"


In [36]:
df.query("price < 3_000_000")

,price,monthly_fee,living_area,number_rooms,type,address
0,2500000,3500,75.5,3,apartment,"Storgatan 12, Stockholm"
2,1800000,2800,50.0,2,apartment,"Lilla Allén 5, Malmö"
4,2100000,4200,65.0,3,apartment,"Norra Parkvägen 7, Lund"
6,1600000,3100,45.0,2,apartment,"Östra Hamngatan 3, Helsingborg"
8,2900000,3800,80.0,3,apartment,"Kungsgatan 1A, Jönköping"
10,2300000,3300,70.0,3,apartment,"Drottninggatan 25, Gävle"
12,1950000,2900,58.0,2,apartment,"Södergatan 9, Västerås"
14,2700000,4000,72.0,3,apartment,"Ringvägen 40, Eskilstuna"
16,1700000,3000,48.0,2,apartment,"Brunnsgatan 15, Karlstad"
18,2400000,3600,68.0,3,apartment,"Nybrogatan 8, Södertälje"
